# Face Recogognition using PCA + SVM

In [1]:
import logging
from PIL import Image

from sklearn.decomposition import PCA
import os,sys
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline

## Read images

In [2]:
path = '/home/ldarmet/Face recognition/CroppedYaleNoisy'
def read_images(path, sz=(168,192)): 
    t0 = time()
    c=0
    
    y = []
    X = np.empty([1, 32256]) #Image are resized to 168x192
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            for filename in os.listdir(subject_path):
                try:
                    print 'Label',c+1,'on :', 37
                    im = Image.open(os.path.join(subject_path , filename)) 
                    im = im.convert("L")
                    # resize to given size (if given)
                    if (sz is not None):
                        im = im.resize(sz, Image.ANTIALIAS)
                    im  = np.asarray(im, dtype=np.uint8)
                    im.flags.writeable = True
                    
                    # Normalization
                    im = (im - im.mean())/im.std()

                    # Make occlusion great again 
                    im[30:60,10:150]=im.min()
                    im[70:100,60:100]=im.min()
                    
                    X = np.vstack((X,im.ravel()))
                    #X.append(np.asarray(im, dtype=np.uint8).ravel())    
                    y.append(c)
                except IOError:
                    print "I/O error({0}): {1}".format("errno", "strerror") #there is some text files in the path
                except:
                    print "Unexpected error:", sys.exc_info()[0] 
                    raise
            c = c+1
        
    y = np.asarray(y)
    X = X[1::,:] # Skip the first line which is void
    print("Image read in %0.3fs" % (time() - t0))
    return [X,y]

## Train test split

In [ ]:
%%time
X,y = read_images(path=path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 on : 37
Label 1 

## Pipeline and CV

Include number of component in PCA to gridsearch 

In [ ]:
%%time
pipe = Pipeline([
    ('reduce_dim', PCA(svd_solver='randomized',
      whiten=True)),
    ('classify', SVC(kernel='rbf', class_weight='balanced'))
])

N_FEATURES_OPTIONS = range(150,250,50)
C_OPTIONS = [1e3, 5e3, 1e4, 5e4, 1e5]
gamma_OPTIONS = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1]
        
param_grid = [
    {
        'reduce_dim__n_components': N_FEATURES_OPTIONS,
        'classify__C': C_OPTIONS,
        'classify__gamma' : gamma_OPTIONS
    }]

grid = GridSearchCV(pipe, cv=3, n_jobs= -1,param_grid=param_grid)
grid = grid.fit(X_train, y_train)
print("Best estimator found by grid search:")
print(grid.best_estimator_)
print grid.best_score_

## Quantitative evaluation of the model quality on the test set

In [ ]:
print("Predicting people's names on the test set")
t0 = time()
y_pred = grid.predict(X_test)
print("done in %0.3fs" % (time() - t0))

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))